### Picks in 1vs1
change if __name__ == "__main__":
    main(territory_count=7, included_picks=[], excluded_picks=[6,8,9])
    
    territory_count -> territories each player receives in the game
    included_picks -> picks you get
    excluded_picks -> picks you lose

In [2]:
from collections import defaultdict

class Node:
    def __init__(self, assigned_picks=None, first_possible=True, second_possible=True):
        self.child_nodes = []
        self.assigned_picks = assigned_picks if assigned_picks is not None else []
        self.first_possible = first_possible
        self.second_possible = second_possible

    def get_copy(self):
        copy_node = Node(self.assigned_picks.copy(), self.first_possible, self.second_possible)
        return copy_node

    def __str__(self):
        picks_str = ", ".join(map(str, self.assigned_picks))
        possible_str = []
        if self.first_possible:
            possible_str.append("first")
        if self.second_possible:
            possible_str.append("second")
        return f"[{picks_str}] ({', '.join(possible_str)})"

def calculate_pick_combinations(parent_node, territory_count):
    if len(parent_node.assigned_picks) == territory_count:
        return

    first_still_possible = parent_node.first_possible
    second_still_possible = parent_node.second_possible
    current_depth = len(parent_node.assigned_picks) + 1

    own_picks = current_depth - 1

    # Player A is picking
    if first_still_possible:
        opponent_picks = current_depth - 1 if current_depth % 2 == 1 else current_depth
        max_pick_number = opponent_picks + 1 + own_picks
        highest_current_pick = parent_node.assigned_picks[-1] if current_depth != 1 else 0

        for i in range(highest_current_pick + 1, max_pick_number + 1):
            child_node = parent_node.get_copy()
            child_node.second_possible = False  # After Player A picks, Player B picks
            child_node.assigned_picks.append(i)
            parent_node.child_nodes.append(child_node)
            calculate_pick_combinations(child_node, territory_count)

    # Player B is picking
    if second_still_possible:
        opponent_picks = current_depth if current_depth % 2 == 1 else current_depth - 1
        max_pick_number = opponent_picks + 1 + own_picks
        highest_current_pick = parent_node.assigned_picks[-1] if current_depth != 1 else 0

        for i in range(highest_current_pick + 1, max_pick_number + 1):
            child_node = parent_node.get_copy()
            child_node.first_possible = False  # After Player B picks, Player A picks
            child_node.assigned_picks.append(i)
            parent_node.child_nodes.append(child_node)
            calculate_pick_combinations(child_node, territory_count)

def get_all_leaves(root):
    leaves = []

    if not root.child_nodes:
        leaves.append(root)
    else:
        for child in root.child_nodes:
            leaves.extend(get_all_leaves(child))

    return leaves

def print_filtered_combinations(root, included_picks=None, excluded_picks=None):
    leaf_nodes = get_all_leaves(root)
    combinations_dict = {}

    if included_picks is None:
        included_picks = set()
    else:
        included_picks = set(included_picks)

    if excluded_picks is None:
        excluded_picks = set()
    else:
        excluded_picks = set(excluded_picks)

    for node in leaf_nodes:
        picks = node.assigned_picks

        # Exclude combinations that contain any of the excluded picks
        if any(pick in excluded_picks for pick in picks):
            continue  # Skip this combination

        # Exclude combinations that do not contain all of the included picks
        if not included_picks.issubset(set(picks)):
            continue  # Skip this combination

        # Group combinations to avoid duplicates
        picks_tuple = tuple(picks)
        if picks_tuple in combinations_dict:
            existing_node = combinations_dict[picks_tuple]
            existing_node.first_possible |= node.first_possible
            existing_node.second_possible |= node.second_possible
        else:
            combinations_dict[picks_tuple] = node

    # Print the combinations
    for node in combinations_dict.values():
        print(node)

def main(territory_count=7, included_picks=None, excluded_picks=None):
    root = Node()
    calculate_pick_combinations(root, territory_count)
    print_filtered_combinations(root, included_picks, excluded_picks)

if __name__ == "__main__":
    main(territory_count=7, included_picks=[], excluded_picks=[6,8,9])


[1, 2, 3, 4, 5, 7, 10] (first, second)
[1, 2, 3, 4, 5, 7, 11] (first, second)
[1, 2, 3, 4, 5, 7, 12] (first, second)
[1, 2, 3, 4, 5, 7, 13] (first, second)
[1, 2, 3, 4, 5, 10, 11] (first, second)
[1, 2, 3, 4, 5, 10, 12] (first, second)
[1, 2, 3, 4, 5, 10, 13] (first, second)
[1, 2, 3, 4, 5, 11, 12] (first, second)
[1, 2, 3, 4, 5, 11, 13] (first, second)
[1, 2, 3, 4, 5, 12, 13] (first)
[1, 2, 3, 4, 7, 10, 11] (first, second)
[1, 2, 3, 4, 7, 10, 12] (first, second)
[1, 2, 3, 4, 7, 10, 13] (first, second)
[1, 2, 3, 4, 7, 11, 12] (first, second)
[1, 2, 3, 4, 7, 11, 13] (first, second)
[1, 2, 3, 4, 7, 12, 13] (first)
[1, 2, 3, 5, 7, 10, 11] (first, second)
[1, 2, 3, 5, 7, 10, 12] (first, second)
[1, 2, 3, 5, 7, 10, 13] (first, second)
[1, 2, 3, 5, 7, 11, 12] (first, second)
[1, 2, 3, 5, 7, 11, 13] (first, second)
[1, 2, 3, 5, 7, 12, 13] (first)
[1, 2, 4, 5, 7, 10, 11] (first, second)
[1, 2, 4, 5, 7, 10, 12] (first, second)
[1, 2, 4, 5, 7, 10, 13] (first, second)
[1, 2, 4, 5, 7, 11, 12] (fir

### 3v3s

change only: def main_included_excluded_3v3():
    total_picks = 12 -> total picks made by team

    teamA_move_order = [1, 4, 5, 8, 9, 12, 13, 16, 17, 20, 21, 24] -> dont change
    teamB_move_order = [2, 3, 6, 7, 10, 11, 14, 15, 18, 19, 22, 23] -> dont change

    # Suppose predetermined locked picks (for locked moves ≤ locked_count).
    # For illustration, we'll use a predetermined_locked list.
    predetermined_locked = []  -> determines whether there are spawns mirrored by both teams. for example if both teams are doing 123 denmark iceland ireland, you can change it to [1,2,3]
    locked_count = len(predetermined_locked)  # 

    # Define the included and excluded picks.
    # For example, suppose you want combinations that must include territory 4 and territory 15,
    # and must exclude territory 9.
    included = []   -> picks in our team
    excluded = [3,4,8,9]  -> picks our team lost

    # Choose a prefix length for grouping (for example, 4)
    prefix_length = 6    -> prefix = 6 gives out ranges like [1,2,5,6,10,11], basically 6-territory combinations we can receive with the above assumptions. 
    for example the above parameters result for team A for these combinations 
        Prefix (1, 2, 5, 6, 7, 10): 1448 combinations (62.8%) -> overall move 12 assigned to A1
        Prefix (1, 2, 5, 6, 7, 11): 647 combinations (28.1%) -> overall move 12 assigned to A1
        Prefix (1, 2, 5, 6, 7, 12): 211 combinations (9.2%) -> overall move 12 assigned to A1
        (basically, can only lose 3,4,8,9 when full mirroring, whenever i get 1,2,5,6,7 when team A)


In [10]:
# --- Mappings for 3v3 based on the provided draft table ---
mapping_A = {
    1: "A1", 4: "A2", 5: "A3",    # Round1
    8: "A3", 9: "A2", 12: "A1",    # Round2
    13: "A1", 16: "A2", 17: "A3",  # Round3
    20: "A3", 21: "A2", 24: "A1"   # Round4
}

mapping_B = {
    2: "B1", 3: "B2", 6: "B3",     # Round1
    7: "B3", 10: "B2", 11: "B1",    # Round2
    14: "B1", 15: "B2", 18: "B3",   # Round3
    19: "B3", 22: "B2", 23: "B1"    # Round4
}


# --- 3v3 Final Combination Generator (locked + free picks) ---
class Node:
    def __init__(self, assigned_picks=None, a_possible=False, b_possible=False):
        # assigned_picks: final increasing sequence of territory numbers (locked picks concatenated with free picks)
        # a_possible: True if this combination can occur for Team A
        # b_possible: True if it can occur for Team B
        self.assigned_picks = assigned_picks if assigned_picks is not None else []
        self.a_possible = a_possible
        self.b_possible = b_possible

    def __str__(self):
        picks_str = ", ".join(map(str, self.assigned_picks))
        poss = []
        if self.a_possible:
            poss.append("A")
        if self.b_possible:
            poss.append("B")
        return f"[{picks_str}] ({', '.join(poss)})"


def calculate_free_combinations(assigned_free, free_team_order, free_opponent_order, index, total_free, base, results):
    """
    Recursively generate valid free pick combinations for a team.
    The allowed maximum for a free pick is:
         max_pick = (# opponent free moves that have occurred) + 1 + (# own free moves so far) + base.
    """
    if index > total_free:
        results.append(assigned_free.copy())
        return

    current_free_move = free_team_order[index - 1]
    opponent_free_count = sum(1 for m in free_opponent_order if m < current_free_move)
    own_free = index - 1
    max_pick = opponent_free_count + 1 + own_free + base
    highest_current = assigned_free[-1] if assigned_free else base

    for i in range(highest_current + 1, max_pick + 1):
        assigned_free.append(i)
        calculate_free_combinations(assigned_free, free_team_order, free_opponent_order, index + 1, total_free, base, results)
        assigned_free.pop()


def calculate_team_with_locked(total_picks, team_move_order, opponent_move_order, locked_count, locked_picks):
    """
    Compute all valid final combinations for a team.
    Final combination = locked picks (drawn from team_move_order positions ≤ locked_count) + free picks.
    """
    locked_team = []
    for m in team_move_order:
        if m <= locked_count:
            # m is 1-indexed; use locked_picks[m - 1]
            locked_team.append(locked_picks[m - 1])
    free_count = total_picks - len(locked_team)
    free_team_order = [m - locked_count for m in team_move_order if m > locked_count]
    free_opponent_order = [m - locked_count for m in opponent_move_order if m > locked_count]
    base = max(locked_picks) if locked_picks else 0
    free_results = []
    calculate_free_combinations([], free_team_order, free_opponent_order, 1, free_count, base, free_results)
    final_combinations = [locked_team + free for free in free_results]
    return final_combinations


# --- Filtering and Grouping Functions ---
def filter_combinations(combos, included=None, excluded=None):
    """
    From a list of final combinations (each a list of territory numbers), filter by:
      - included: a set (or list) of territory numbers that must appear.
      - excluded: a set (or list) of territory numbers that must not appear.
    """
    if included is None:
        included = set()
    else:
        included = set(included)
    if excluded is None:
        excluded = set()
    else:
        excluded = set(excluded)

    filtered = []
    for combo in combos:
        if included and not included.issubset(set(combo)):
            continue
        if excluded and any(x in combo for x in excluded):
            continue
        filtered.append(combo)
    return filtered


def group_by_prefix(combos, prefix_length):
    """
    Group a list of combinations by their prefix (first prefix_length picks).
    Returns a dictionary mapping prefix (tuple) -> count.
    """
    groups = {}
    for combo in combos:
        if len(combo) >= prefix_length:
            prefix = tuple(combo[:prefix_length])
            groups.setdefault(prefix, 0)
            groups[prefix] += 1
    return groups


def print_grouped_results(groups, total, move_order, mapping, prefix_length):
    """
    Print the grouped results.
    For each prefix, report:
       - The prefix,
       - The overall move corresponding to the prefix's last pick,
       - Which team–player that overall move is assigned to,
       - The count and percentage.
    """
    for prefix, count in sorted(groups.items()):
        overall_move = move_order[prefix_length - 1]  # the overall move for the prefix's last pick
        assigned_player = mapping.get(overall_move, "Unknown")
        percentage = count / total * 100
        print(f"Prefix {prefix}: {count} combinations ({percentage:.1f}%) -> overall move {overall_move} assigned to {assigned_player}")


def analyze_included_excluded_for_role(role, total_picks, teamA_move_order, teamB_move_order,
                                       locked_count, predetermined_locked,
                                       included=None, excluded=None, prefix_length=4):
    """
    Generate all valid final combinations for a given team (role in 3v3), filter them by the included
    and excluded picks, then group the filtered combinations by the prefix of length prefix_length.
    
    The function prints:
       - The total number of valid final combinations (before filtering),
       - The number remaining after filtering,
       - And, grouped by prefix, for each group:
             * The prefix,
             * The overall move (from the team's move order) corresponding to that pick,
             * Which team–player gets that move,
             * The count and percentage.
    """
    if role.upper() == 'A':
        combos = calculate_team_with_locked(total_picks, teamA_move_order, teamB_move_order, locked_count, predetermined_locked)
        move_order = teamA_move_order
        mapping = mapping_A
    elif role.upper() == 'B':
        combos = calculate_team_with_locked(total_picks, teamB_move_order, teamA_move_order, locked_count, predetermined_locked)
        move_order = teamB_move_order
        mapping = mapping_B
    else:
        raise ValueError("Role must be 'A' or 'B'")
    
    total_combos = len(combos)
    filtered = filter_combinations(combos, included, excluded)
    filtered_total = len(filtered)
    print(f"\nTeam {role}:")
    print(f"Total valid final combinations: {total_combos}")
    print(f"After filtering (included: {included}, excluded: {excluded}): {filtered_total}")
    
    if filtered_total == 0:
        return
    
    groups = group_by_prefix(filtered, prefix_length)
    print(f"\nGrouping by first {prefix_length} picks:")
    print_grouped_results(groups, filtered_total, move_order, mapping, prefix_length)


###############################################################################
# Example main function for included/excluded analysis in 3v3.
###############################################################################

def main_included_excluded_3v3():
    # For a 3v3 game, assume each team gets 12 picks. Assume full mirror in each team
    total_picks = 12

    # Overall move orders based on the provided draft table:
    teamA_move_order = [1, 4, 5, 8, 9, 12, 13, 16, 17, 20, 21, 24]
    teamB_move_order = [2, 3, 6, 7, 10, 11, 14, 15, 18, 19, 22, 23]

    # Suppose predetermined locked picks (for locked moves ≤ locked_count).
    # For illustration, we'll use a predetermined_locked list.
    predetermined_locked = []
    locked_count = len(predetermined_locked)  # 

    # Define the included and excluded picks.
    # For example, suppose you want combinations that must include territory 4 and territory 15,
    # and must exclude territory 9.
    included = []
    excluded = [3,4,8,9]

    # Choose a prefix length for grouping (for example, 4).
    prefix_length = 6

    # Analyze for Team A:
    print("=== Analysis for Team A (with included/excluded picks) ===")
    analyze_included_excluded_for_role("A", total_picks, teamA_move_order, teamB_move_order,
                                       locked_count, predetermined_locked,
                                       included, excluded, prefix_length)
    # Analyze for Team B:
    print("\n=== Analysis for Team B (with included/excluded picks) ===")
    analyze_included_excluded_for_role("B", total_picks, teamA_move_order, teamB_move_order,
                                       locked_count, predetermined_locked,
                                       included, excluded, prefix_length)


if __name__ == "__main__":
    main_included_excluded_3v3()


=== Analysis for Team A (with included/excluded picks) ===

Team A:
Total valid final combinations: 338444
After filtering (included: [], excluded: [3, 4, 8, 9]): 2306

Grouping by first 6 picks:
Prefix (1, 2, 5, 6, 7, 10): 1448 combinations (62.8%) -> overall move 12 assigned to A1
Prefix (1, 2, 5, 6, 7, 11): 647 combinations (28.1%) -> overall move 12 assigned to A1
Prefix (1, 2, 5, 6, 7, 12): 211 combinations (9.2%) -> overall move 12 assigned to A1

=== Analysis for Team B (with included/excluded picks) ===

Team B:
Total valid final combinations: 338444
After filtering (included: [], excluded: [3, 4, 8, 9]): 3414

Grouping by first 6 picks:
Prefix (1, 2, 5, 6, 7, 10): 1198 combinations (35.1%) -> overall move 11 assigned to B1
Prefix (1, 2, 5, 6, 7, 11): 554 combinations (16.2%) -> overall move 11 assigned to B1
Prefix (1, 2, 5, 6, 10, 11): 554 combinations (16.2%) -> overall move 11 assigned to B1
Prefix (1, 2, 5, 7, 10, 11): 554 combinations (16.2%) -> overall move 11 assigned t

## Biomes 2vs2 

8 territories per team, 4 per player



In [ ]:
class Node:
    def __init__(self, assigned_picks=None, a_possible=False, b_possible=False):
        # assigned_picks: final increasing sequence of territory numbers (locked + free)
        # a_possible: True if this combination is valid for Team A
        # b_possible: True if valid for Team B
        self.assigned_picks = assigned_picks if assigned_picks is not None else []
        self.a_possible t= a_possible
        self.b_possible = b_possible

    def __str__(self):
        picks_str = ", ".join(map(str, self.assigned_picks))
        poss = []
        if self.a_possible:
            poss.append("A")
        if self.b_possible:
            poss.append("B")
        return f"[{picks_str}] ({', '.join(poss)})"


def calculate_free_combinations(assigned_free, free_team_order, free_opponent_order, index, total_free, base, results):
    """
    Recursively generate valid free pick combinations for a team.
    - free_team_order: adjusted free move order (each value = original move – locked_count)
    - free_opponent_order: opponent’s free move order (adjusted similarly)
    - base: the common base for free picks, set to max(locked_picks) so free picks > base.
    The allowed maximum for a free pick is:
         max_pick = (# opponent free moves that have occurred) + 1 + (# own free moves so far) + base.
    """
    if index > total_free:
        results.append(assigned_free.copy())
        return

    current_free_move = free_team_order[index - 1]
    opponent_free_count = sum(1 for m in free_opponent_order if m < current_free_move)
    own_free = index - 1
    max_pick = opponent_free_count + 1 + own_free + base

    highest_current = assigned_free[-1] if assigned_free else base

    for i in range(highest_current + 1, max_pick + 1):
        assigned_free.append(i)
        calculate_free_combinations(assigned_free, free_team_order, free_opponent_order, index + 1, total_free, base, results)
        assigned_free.pop()


def calculate_team_with_locked(total_picks, team_move_order, opponent_move_order, locked_count, locked_picks):
    """
    Compute valid final pick combinations for a team when a predetermined set of locked picks is used.
    
    Parameters:
      total_picks: total number of picks for the team.
      team_move_order: overall move numbers when this team picks.
      opponent_move_order: overall move numbers for the opponent.
      locked_count: number of predetermined (locked) moves.
      locked_picks: list of predetermined territory numbers (common to both teams) in overall order.
    
    Returns a list of final combinations (each is locked picks for that team + free picks).
    The locked picks for this team are the ones from team_move_order that are ≤ locked_count.
    The free picks are generated using a base equal to max(locked_picks), so free picks will be > max(locked_picks).
    """
    # Determine locked picks for this team.
    locked_team = []
    for m in team_move_order:
        if m <= locked_count:
            # m is 1-indexed; use locked_picks[m - 1]
            locked_team.append(locked_picks[m - 1])
    free_count = total_picks - len(locked_team)

    # Adjust free moves: only moves > locked_count and subtract locked_count.
    free_team_order = [m - locked_count for m in team_move_order if m > locked_count]
    free_opponent_order = [m - locked_count for m in opponent_move_order if m > locked_count]

    # Set the base to the maximum of the predetermined locked picks (so free picks are > all locked picks).
    base = max(locked_picks) if locked_picks else 0

    free_results = []
    calculate_free_combinations([], free_team_order, free_opponent_order, 1, free_count, base, free_results)

    # Merge locked picks with each free combination.
    final_combinations = [locked_team + free for free in free_results]
    return final_combinations


def main():
    # For 8 picks per team, you need overall move orders of length 8.
    # Here’s one possible move order (example only):
    teamA_move_order = [1, 4, 5, 8, 9, 12, 13, 16]
    teamB_move_order = [2, 3, 6, 7, 10, 11, 14, 15]

    # Suppose both teams have predetermined top picks (locked picks) that are the same.
    predetermined_locked = [1, 2, 3]  # common list (in overall order)
    locked_count = len(predetermined_locked)

    # Set total picks per team to 8
    total_picks_A = 8
    total_picks_B = 8

    # Compute combinations for each team.
    combos_A = calculate_team_with_locked(total_picks_A, teamA_move_order, teamB_move_order, locked_count, predetermined_locked)
    combos_B = calculate_team_with_locked(total_picks_B, teamB_move_order, teamA_move_order, locked_count, predetermined_locked)

    # Merge results from both teams, marking possibility flags.
    merged = {}
    for combo in combos_A:
        key = tuple(combo)
        if key in merged:
            merged[key].a_possible = True
        else:
            merged[key] = Node(combo.copy(), a_possible=True, b_possible=False)
    for combo in combos_B:
        key = tuple(combo)
        if key in merged:
            merged[key].b_possible = True
        else:
            merged[key] = Node(combo.copy(), a_possible=False, b_possible=True)

    print("Valid final pick combinations with predetermined locked picks (common top picks):")
    for node in merged.values():
        print(node)


if __name__ == "__main__":
    main()


Valid final pick combinations with predetermined locked picks (common top picks):
[1, 4, 5, 6, 7, 8, 9, 10] (A)
[1, 4, 5, 6, 7, 8, 9, 11] (A)
[1, 4, 5, 6, 7, 8, 9, 12] (A)
[1, 4, 5, 6, 7, 8, 9, 13] (A)
[1, 4, 5, 6, 7, 8, 9, 14] (A)
[1, 4, 5, 6, 7, 8, 9, 15] (A)
[1, 4, 5, 6, 7, 8, 9, 16] (A)
[1, 4, 5, 6, 7, 8, 10, 11] (A)
[1, 4, 5, 6, 7, 8, 10, 12] (A)
[1, 4, 5, 6, 7, 8, 10, 13] (A)
[1, 4, 5, 6, 7, 8, 10, 14] (A)
[1, 4, 5, 6, 7, 8, 10, 15] (A)
[1, 4, 5, 6, 7, 8, 10, 16] (A)
[1, 4, 5, 6, 7, 8, 11, 12] (A)
[1, 4, 5, 6, 7, 8, 11, 13] (A)
[1, 4, 5, 6, 7, 8, 11, 14] (A)
[1, 4, 5, 6, 7, 8, 11, 15] (A)
[1, 4, 5, 6, 7, 8, 11, 16] (A)
[1, 4, 5, 6, 7, 8, 12, 13] (A)
[1, 4, 5, 6, 7, 8, 12, 14] (A)
[1, 4, 5, 6, 7, 8, 12, 15] (A)
[1, 4, 5, 6, 7, 8, 12, 16] (A)
[1, 4, 5, 6, 7, 8, 13, 14] (A)
[1, 4, 5, 6, 7, 8, 13, 15] (A)
[1, 4, 5, 6, 7, 8, 13, 16] (A)
[1, 4, 5, 6, 7, 9, 10, 11] (A)
[1, 4, 5, 6, 7, 9, 10, 12] (A)
[1, 4, 5, 6, 7, 9, 10, 13] (A)
[1, 4, 5, 6, 7, 9, 10, 14] (A)
[1, 4, 5, 6, 7, 9, 10, 15]

## Final Earth 2vs2

6 territories per team

In [2]:
class Node:
    def __init__(self, assigned_picks=None, a_possible=False, b_possible=False):
        # assigned_picks: final increasing sequence of territory numbers (locked + free)
        # a_possible: True if this combination is valid for Team A
        # b_possible: True if valid for Team B
        self.assigned_picks = assigned_picks if assigned_picks is not None else []
        self.a_possible = a_possible
        self.b_possible = b_possible

    def __str__(self):
        picks_str = ", ".join(map(str, self.assigned_picks))
        poss = []
        if self.a_possible:
            poss.append("A")
        if self.b_possible:
            poss.append("B")
        return f"[{picks_str}] ({', '.join(poss)})"


def calculate_free_combinations(assigned_free, free_team_order, free_opponent_order, index, total_free, base, results):
    """
    Recursively generate valid free pick combinations for a team.
    - free_team_order: adjusted free move order (each value = original move – locked_count)
    - free_opponent_order: opponent’s free move order (adjusted similarly)
    - base: the common base for free picks, set to max(locked_picks) so free picks > base.
    The allowed maximum for a free pick is:
         max_pick = (# opponent free moves that have occurred) + 1 + (# own free moves so far) + base.
    """
    if index > total_free:
        results.append(assigned_free.copy())
        return

    current_free_move = free_team_order[index - 1]
    opponent_free_count = sum(1 for m in free_opponent_order if m < current_free_move)
    own_free = index - 1
    max_pick = opponent_free_count + 1 + own_free + base

    highest_current = assigned_free[-1] if assigned_free else base

    for i in range(highest_current + 1, max_pick + 1):
        assigned_free.append(i)
        calculate_free_combinations(assigned_free, free_team_order, free_opponent_order, index + 1, total_free, base, results)
        assigned_free.pop()


def calculate_team_with_locked(total_picks, team_move_order, opponent_move_order, locked_count, locked_picks):
    """
    Compute valid final pick combinations for a team when a predetermined set of locked picks is used.
    
    Parameters:
      total_picks: total number of picks for the team.
      team_move_order: overall move numbers when this team picks.
      opponent_move_order: overall move numbers for the opponent.
      locked_count: number of predetermined (locked) moves.
      locked_picks: list of predetermined territory numbers (common to both teams) in overall order.
    
    Returns a list of final combinations (each is locked picks for that team + free picks).
    The locked picks for this team are the ones from team_move_order that are ≤ locked_count.
    The free picks are generated using a base equal to max(locked_picks), so free picks will be > max(locked_picks).
    """
    # Determine locked picks for this team.
    locked_team = []
    for m in team_move_order:
        if m <= locked_count:
            # m is 1-indexed; use locked_picks[m - 1]
            locked_team.append(locked_picks[m - 1])
    free_count = total_picks - len(locked_team)

    # Adjust free moves: only moves > locked_count and subtract locked_count.
    free_team_order = [m - locked_count for m in team_move_order if m > locked_count]
    free_opponent_order = [m - locked_count for m in opponent_move_order if m > locked_count]

    # Set the base to the maximum of the predetermined locked picks (so free picks are > all locked picks).
    base = max(locked_picks) if locked_picks else 0

    free_results = []
    calculate_free_combinations([], free_team_order, free_opponent_order, 1, free_count, base, free_results)

    # Merge locked picks with each free combination.
    final_combinations = [locked_team + free for free in free_results]
    return final_combinations


def main():
    # Example parameters:
    # Overall move orders for a 2v2 game (from your corrected table):
    #         | Round1 | Round2 | Round3
    # A1      |   1    |    8   |   9
    # B1      |   2    |    7   |  10
    # B2      |   3    |    6   |  11
    # A2      |   4    |    5   |  12
    teamA_move_order = [1, 4, 5, 8, 9, 12]
    teamB_move_order = [2, 3, 6, 7, 10, 11]

    # Suppose both teams have predetermined top picks (locked picks) that are the same.
    predetermined_locked = [1, 2, 3, 4, 5]  # common list (in overall order)
    locked_count = len(predetermined_locked)

    # For demonstration, let’s assume the final total picks per team are as follows:
    # (In your verbal example, you mentioned:
    #    Team A gets locked: [1,4,5]
    #    Team B gets locked: [2,3] and then later picks “6/7”)
    # Here we set total picks so that:
    #  - Team A ends up with 3 picks total (all locked in this example).
    #  - Team B ends up with 4 picks total (2 locked + 2 free).
    total_picks_A = 6
    total_picks_B = 6

    # Compute combinations for each team.
    combos_A = calculate_team_with_locked(total_picks_A, teamA_move_order, teamB_move_order, locked_count, predetermined_locked)
    combos_B = calculate_team_with_locked(total_picks_B, teamB_move_order, teamA_move_order, locked_count, predetermined_locked)

    # Merge results from both teams, marking possibility flags.
    merged = {}
    for combo in combos_A:
        key = tuple(combo)
        if key in merged:
            merged[key].a_possible = True
        else:
            merged[key] = Node(combo.copy(), a_possible=True, b_possible=False)
    for combo in combos_B:
        key = tuple(combo)
        if key in merged:
            merged[key].b_possible = True
        else:
            merged[key] = Node(combo.copy(), a_possible=False, b_possible=True)

    print("Valid final pick combinations with predetermined locked picks (common top picks):")
    for node in merged.values():
        print(node)


if __name__ == "__main__":
    main()


Valid final pick combinations with predetermined locked picks (common top picks):
[1, 4, 5, 6, 7, 8] (A)
[1, 4, 5, 6, 7, 9] (A)
[1, 4, 5, 6, 7, 10] (A)
[1, 4, 5, 6, 7, 11] (A)
[1, 4, 5, 6, 7, 12] (A)
[1, 4, 5, 6, 8, 9] (A)
[1, 4, 5, 6, 8, 10] (A)
[1, 4, 5, 6, 8, 11] (A)
[1, 4, 5, 6, 8, 12] (A)
[1, 4, 5, 6, 9, 10] (A)
[1, 4, 5, 6, 9, 11] (A)
[1, 4, 5, 6, 9, 12] (A)
[1, 4, 5, 7, 8, 9] (A)
[1, 4, 5, 7, 8, 10] (A)
[1, 4, 5, 7, 8, 11] (A)
[1, 4, 5, 7, 8, 12] (A)
[1, 4, 5, 7, 9, 10] (A)
[1, 4, 5, 7, 9, 11] (A)
[1, 4, 5, 7, 9, 12] (A)
[1, 4, 5, 8, 9, 10] (A)
[1, 4, 5, 8, 9, 11] (A)
[1, 4, 5, 8, 9, 12] (A)
[2, 3, 6, 7, 8, 9] (B)
[2, 3, 6, 7, 8, 10] (B)
[2, 3, 6, 7, 8, 11] (B)
[2, 3, 6, 7, 9, 10] (B)
[2, 3, 6, 7, 9, 11] (B)
[2, 3, 6, 7, 10, 11] (B)


### Final Earth 2vs2 with analysis for missing picks

In [ ]:
from collections import defaultdict

# --- 2v2 Combination Generator (locked + free picks) ---
class Node:
    def __init__(self, assigned_picks=None, a_possible=False, b_possible=False):
        # assigned_picks: finttal increasing sequence of territory numbers 
        # (locked picks concatenated with free picks)
        # a_possible: True if this combination can occur for Team A
        # b_possible: True if it can occur for Team B
        self.assigned_picks = assigned_picks if assigned_picks is not None else []
        self.a_possible = a_possible
        self.b_possible = b_possible

    def __str__(self):
        picks_str = ", ".join(map(str, self.assigned_picks))
        poss = []
        if self.a_possible:
            poss.append("A")
        if self.b_possible:
            poss.append("B")
        return f"[{picks_str}] ({', '.join(poss)})"

def calculate_free_combinations(assigned_free, free_team_order, free_opponent_order, index, total_free, base, results):
    """
    Recursively generate valid free pick combinations for a team.
    Allowed maximum for a free pick:
         max_pick = (# opponent free moves occurred) + 1 + (# own free moves so far) + base.
    """
    if index > total_free:
        results.append(assigned_free.copy())
        return

    current_free_move = free_team_order[index - 1]
    opponent_free_count = sum(1 for m in free_opponent_order if m < current_free_move)
    own_free = index - 1
    max_pick = opponent_free_count + 1 + own_free + base
    highest_current = assigned_free[-1] if assigned_free else base

    for i in range(highest_current + 1, max_pick + 1):
        assigned_free.append(i)
        calculate_free_combinations(assigned_free, free_team_order, free_opponent_order, index + 1, total_free, base, results)
        assigned_free.pop()

def calculate_team_with_locked(total_picks, team_move_order, opponent_move_order, locked_count, locked_picks):
    """
    Compute all valid final combinations for a team.
    Final combination = locked picks (from team_move_order positions ≤ locked_count) + free picks.
    """
    locked_team = []
    for m in team_move_order:
        if m <= locked_count:
            # m is 1-indexed; use locked_picks[m - 1]
            locked_team.append(locked_picks[m - 1])
    free_count = total_picks - len(locked_team)
    free_team_order = [m - locked_count for m in team_move_order if m > locked_count]
    free_opponent_order = [m - locked_count for m in opponent_move_order if m > locked_count]
    base = max(locked_picks) if locked_picks else 0
    free_results = []
    calculate_free_combinations([], free_team_order, free_opponent_order, 1, free_count, base, free_results)
    final_combinations = [locked_team + free for free in free_results]
    return final_combinations

# --- Filtering and Grouping Functions for 2v2 Analysis ---
def filter_combinations(combos, included=None, excluded=None):
    """
    From a list of final combinations (each a list of territory numbers), filter by:
      - included: a set (or list) of territory numbers that must appear.
      - excluded: a set (or list) of territory numbers that must not appear.
    """
    if included is None:
        included = set()
    else:
        included = set(included)
    if excluded is None:
        excluded = set()
    else:
        excluded = set(excluded)

    filtered = []
    for combo in combos:
        if included and not included.issubset(set(combo)):
            continue
        if excluded and any(x in combo for x in excluded):
            continue
        filtered.append(combo)
    return filtered

def group_by_prefix(combos, prefix_length):
    """
    Group a list of combinations by their prefix (first prefix_length picks).
    Returns a dictionary mapping prefix (tuple) -> count.
    """
    groups = {}
    for combo in combos:
        if len(combo) >= prefix_length:
            prefix = tuple(combo[:prefix_length])
            groups.setdefault(prefix, 0)
            groups[prefix] += 1
    return groups

def print_grouped_results(groups, total, move_order, mapping, prefix_length):
    """
    For each prefix group, report:
       - The prefix,
       - The overall move corresponding to the prefix's last pick (from move_order),
       - Which team–player that overall move is assigned to (from mapping),
       - The count and percentage.
    """
    for prefix, count in sorted(groups.items()):
        overall_move = move_order[prefix_length - 1]
        assigned_player = mapping.get(overall_move, "Unknown")
        percentage = count / total * 100
        print(f"Prefix {prefix}: {count} combinations ({percentage:.1f}%) -> overall move {overall_move} assigned to {assigned_player}")

def analyze_included_excluded_for_role(role, total_picks, teamA_move_order, teamB_move_order,
                                       locked_count, predetermined_locked,
                                       included=None, excluded=None, prefix_length=3):
    """
    For a 2v2 team, generate all valid final combinations (using locked+free picks),
    filter them by the included/excluded territory lists, group by the prefix of length prefix_length,
    and print:
       - Total valid combinations,
       - Number remaining after filtering,
       - For each group: the prefix, the overall move for its last pick, the assigned player, and the count/percentage.
    """
    if role.upper() == 'A':
        combos = calculate_team_with_locked(total_picks, teamA_move_order, teamB_move_order, locked_count, predetermined_locked)
        move_order = teamA_move_order
        # For 2v2, we define a mapping based on a snake draft between two players.
        # For instance, for Team A's overall move order [1,4,5,8,9,12]:
        mapping = {1: "A1", 4: "A2", 5: "A2", 8: "A1", 9: "A1", 12: "A2"}
    elif role.upper() == 'B':
        combos = calculate_team_with_locked(total_picks, teamB_move_order, teamA_move_order, locked_count, predetermined_locked)
        move_order = teamB_move_order
        mapping = {2: "B1", 3: "B2", 6: "B2", 7: "B1", 10: "B1", 11: "B2"}
    else:
        raise ValueError("Role must be 'A' or 'B'")

    total_combos = len(combos)
    filtered = filter_combinations(combos, included, excluded)
    filtered_total = len(filtered)
    print(f"\nTeam {role}:")
    print(f"Total valid final combinations: {total_combos}")
    print(f"After filtering (included: {included}, excluded: {excluded}): {filtered_total}")

    if filtered_total == 0:
        return

    groups = group_by_prefix(filtered, prefix_length)
    print(f"\nGrouping by first {prefix_length} picks:")
    print_grouped_results(groups, filtered_total, move_order, mapping, prefix_length)

###############################################################################
# Example main function for 2v2 included/excluded analysis.
###############################################################################
def main_included_excluded_2v2():
    # For a 2v2 game, assume each team gets 6 picks.
    total_picks = 6

    # Overall move orders based on a 2v2 draft table:
    teamA_move_order = [1, 4, 5, 8, 9, 12]
    teamB_move_order = [2, 3, 6, 7, 10, 11]

    # Suppose predetermined locked picks are defined.
    # For example, let predetermined_locked = [1, 2, 3, 4, 5]
    # For Team A, locked picks are those from teamA_move_order ≤ locked_count.
    # Here, locked_count = len(predetermined_locked) = 5.
    predetermined_locked = [1,2]
    locked_count = len(predetermined_locked)

    # Define included and excluded picks.
    # For instance, suppose we want to include territory 1 and exclude territory 3.
    included = []
    excluded = [6,7]

    # Choose a prefix length for grouping analysis. For a 6–pick combination, try prefix_length = 3.
    prefix_length = 5

    # Analyze for Team A:
    print("=== Analysis for 2v2 (Team A, with included/excluded picks) ===")
    analyze_included_excluded_for_role("A", total_picks, teamA_move_order, teamB_move_order,
                                       locked_count, predetermined_locked,
                                       included, excluded, prefix_length)

    # Analyze for Team B:
    print("\n=== Analysis for 2v2 (Team B, with included/excluded picks) ===")
    analyze_included_excluded_for_role("B", total_picks, teamA_move_order, teamB_move_order,
                                       locked_count, predetermined_locked,
                                       included, excluded, prefix_length)

if __name__ == "__main__":
    main_included_excluded_2v2()


=== Analysis for 2v2 (Team A, with included/excluded picks) ===

Team A:
Total valid final combinations: 84
After filtering (included: [], excluded: [6, 7]): 16

Grouping by first 5 picks:
Prefix (1, 3, 4, 5, 8): 4 combinations (25.0%) -> overall move 9 assigned to A1
Prefix (1, 3, 4, 5, 9): 3 combinations (18.8%) -> overall move 9 assigned to A1
Prefix (1, 3, 4, 8, 9): 3 combinations (18.8%) -> overall move 9 assigned to A1
Prefix (1, 3, 5, 8, 9): 3 combinations (18.8%) -> overall move 9 assigned to A1
Prefix (1, 4, 5, 8, 9): 3 combinations (18.8%) -> overall move 9 assigned to A1

=== Analysis for 2v2 (Team B, with included/excluded picks) ===

Team B:
Total valid final combinations: 53
After filtering (included: [], excluded: [6, 7]): 6

Grouping by first 5 picks:
Prefix (2, 3, 4, 5, 8): 3 combinations (50.0%) -> overall move 10 assigned to B1
Prefix (2, 3, 4, 5, 9): 2 combinations (33.3%) -> overall move 10 assigned to B1
Prefix (2, 3, 4, 5, 10): 1 combinations (16.7%) -> overall m